# Section ID Count and Event Data
- limited to WP:M pages

Section ID data was not limited to top-level (H2) sections during data capture, requiring post-capture processing for section ID click data. Post-processing included scraping section data from HTML pages. There is a potential for data loss since scraping occured after click capture. Only WPM pages were scraped on 2019-04-23 PDT, meaning no comparison to W pages is possible. Extracting section data from dump files is another option, although recreating section IDs from text is problematic, making the approach here preferred. 

Example of capture issue: https://en.wikipedia.org/wiki/Hepatitis#Signs_and_symptoms.
Clicks on links under "Acute hepatitis" were captured with section_id Acute_hepatitis, not Signs_and_symptoms.

Post-capture data augmentation: H2 sections were extracted from public HTML of WPM pages 2019-04-23

See [populate-section-table.ipynb](populate-section-table.ipynb) for extraction details


In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'

## Count of top-level (H2) section IDs for WP:M pages only


In [2]:
# count of top-level (H2) section IDs for WP:M pages only
pm_sections_query = """
SELECT section_h2, count(*) count
FROM
    ryanmax.wpm_sections 
GROUP BY section_h2
ORDER BY count desc, section_h2
"""
pm_sections = spark.sql(pm_sections_query)
pm_sections.show()

+-------------------+-----+
|         section_h2|count|
+-------------------+-----+
|         References|31533|
|     External_links|18683|
|           See_also|13482|
|          Diagnosis| 8625|
|            History| 8120|
|          Treatment| 7245|
| Signs_and_symptoms| 4657|
|             Causes| 4413|
|    Further_reading| 3284|
|Society_and_culture| 3055|
|       Epidemiology| 2653|
|       Medical_uses| 2508|
|              Cause| 2151|
|         Management| 2057|
|           Research| 2031|
|              Types| 1989|
|    Pathophysiology| 1897|
|             Career| 1805|
|       Pharmacology| 1634|
|          Prognosis| 1539|
+-------------------+-----+
only showing top 20 rows



## Total count of events (all types) for each top-level (H2) section ID for WP:M pages only
- Limited to >= 3000 events
- **missing** values are largely because section IDs were not recorded "if the section is the Main Section" as per [Schema:CitationUsage](https://meta.wikimedia.org/wiki/Schema:CitationUsage).
- Numbers may not exactly match "raw" data (below) because some section IDs could not be mapped to H2 section IDs (changed section ID, missing WPM page, etc.)


In [3]:
# Total count of events (by all event types) for each top-level (H2) section ID for WP:M pages only
pm_section_events_query = """
SELECT wpm_sections.section_h2, action, count(*) count
FROM 
    citationusage
    LEFT JOIN ryanmax.wpm_sections 
        ON 
        wpm_sections.page_id = citationusage.page_id 
        AND wpm_sections.section_id = citationusage.section_id
WHERE
    wiki = 'enwiki'
    AND citationusage.page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    {}
    AND to_date(citationusage.event_time) >= '{}'
    AND to_date(citationusage.event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY wpm_sections.section_h2, action
ORDER BY count desc
"""
pm_section_events = spark.sql(
    pm_section_events_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_rdd = pm_section_events.rdd
pm_section_events_df = sqlContext.createDataFrame(pm_section_events_rdd)
pm_section_events_pandas = pm_section_events_df.toPandas()

In [4]:
section_pda = pm_section_events_pandas.copy()
# replace 'NaN' section_h2 with 'missing'
section_pda.section_h2.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered = section_pda.query('count>3000').copy()
# set precision before pivot
df_filtered['count'] = df_filtered['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered.pivot(index='section_h2', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,143808,367433,527370,None
Adverse_effects,None,8014,14141,None
Background,None,None,4216,None
Biography,None,None,3864,None
Career,None,None,3878,None
Cause,None,8959,21897,None
Causes,None,20475,39955,None
Characteristics,None,None,3835,None
Chemistry,None,None,4100,None


## Raw section data from captured events
- Raw total count of events (by all event types) for each section ID for WP:M pages only.
- Limited to >= 3000 events.
- shows the extent of the capture issue described above


In [5]:
# "raw" section data from captured events to show extent of the capture issue described above
pm_section_events_raw_query = """
SELECT section_id, action, count(*) count
FROM 
    citationusage 
WHERE page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    AND wiki = 'enwiki'
    {}
    AND to_date(event_time) >= '{}'
    AND to_date(event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY section_id, action
ORDER BY count desc
LIMIT 100
"""
pm_section_events_raw = spark.sql(
    pm_section_events_raw_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_raw_rdd = pm_section_events_raw.rdd
pm_section_events_raw_df = sqlContext.createDataFrame(pm_section_events_raw_rdd)
pm_section_events_raw_pandas = pm_section_events_raw_df.toPandas()
#pm_section_events_raw_pandas
#pm_section_events_pandas.pivot(index='section_id', columns='action', values='count')

In [6]:
section_pda_raw = pm_section_events_raw_pandas.copy()
# replace 'NaN' section_id with 'missing'
section_pda_raw.section_id.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered_raw = section_pda_raw.query('count>3000').copy()
# set precision before pivot
df_filtered_raw['count'] = df_filtered_raw['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered_raw.pivot(index='section_id', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
section_id,,,,
-- missing --,143490,365209,523476,None
Adverse_effects,None,4252,6343,None
Background,None,None,3496,None
Cause,None,3821,7787,None
Causes,None,10930,18686,None
Classification,None,3029,6620,None
Criminal_charges,None,3025,None,None
Diagnosis,None,7353,14525,None
Downfall,None,None,3523,None
